# Линейная регрессия

Здесь хороший повод поговорить о математике, но меня просили не. Что ж, надеюсь, это не помешает вам сопоставить теорию с практикой. 

Суть: у нас есть несколько параметров и есть некоторое числовое значение, зависимость которого от этих самых параметров мы и хотим узнать. Например: есть количество квартир в доме, есть площадь парковки возле дома и есть площадь конкретной квартиры. По этим трём параметрам нужно найти её стоимость. Пара из трёх параметров и цены квартиры составляют одну запись/один индивид. Таких индивидов у нас несколько (желательно довольно много). 

## Цели линейной регрессии

Собственно линейную регрессию используют для двух целей:
* Предсказать значение нового индивида при известных параметрах
* Объяснить как влияет каждый из параметров на искомое значение